In [1]:
# Import Gurobi

from gurobipy import *
import numpy as np

In [2]:
# Create the model

M = Model(name = 'PI 1');

Academic license - for non-commercial use only


In [3]:
# Team names

names = ['Independiente Medellín', 'Atlético Nacional',
         'Deportivo Cali',         'América de Cali',
         'La Equidad',             'Patriotas',
         'Envigado F.C',           'Rionegro Águilas',
         'Once Caldas',            'Deportivo Pasto',
         'Unión Magdalena',        'Junior',
         'Atlético Huila',         'Deportes Tolima',
         'Jaguares',               'Alianza Petrolera',
         'Atlético Bucaramanga',   'Cúcuta Deportivo',
         'Millonarios',            'Santa Fe']

In [4]:
# Model parameters

# Number of teams
n = 20
T = [i for i in range(n)]

# Number of dates
m = 30
P = [i for i in range(m)]

# Date of non classical matches
c = 15
Q = [i for i in range(m) if i != c]

# Team-pairs that share stadium
S = [(0, 1), (2, 3), (18, 19)]

# Classic matches
C = [(i, i + 1) for i in range(0, n - 1, 2)]

# Prohibited dates for some teams (due to other competitions)
F = {

    # Copa sudamericana
    2 : [21, 29],               # Deportivo Cali
    4 : [17, 25],               # La Equidad
    7 : [17, 25],               # Rionegro Águilas
    8 : [5, 9],                 # Once Caldas

    # Libertadores
    0  : [5, 7],                # Independiente Medellín
    1  : [5, 7, 9, 11],         # Atlético Nacional
    11 : [13, 15, 21, 23, 27],  # Junior
    13 : [13, 15, 21, 23, 27],  # Deportes Tolima

    # Copa Colombia
    3  : [7, 15, 18, 23, 27],   # América de Cali
    5  : [7, 15, 18, 23, 27],   # Patriotas
    6  : [7, 15, 18, 23, 27],   # Envigado
    9  : [7, 15, 18, 23, 27],   # Deportivo Pasto
    10 : [15, 18, 21, 23, 27],  # Unión Magdalena
    12 : [7, 15, 18, 23, 27],   # Atlético Huila
    14 : [7, 15, 18, 23, 27],   # Jaguares
    15 : [7, 15, 18, 23, 27],   # Alianza Petrolera
    16 : [7, 15, 18, 23, 27],   # Atlético Bucaramanga
    17 : [7, 15, 18, 23, 27],   # Cúcuta Deportivo
    18 : [7, 15, 18, 23, 27],   # Millonarios
    19 : [7, 15, 18, 23, 27]    # Santa Fe
}

F2 = F.copy()
for key in F2.keys():
    dates = np.zeros(m)
    for value in F2[key]:
        dates[value - 1] = 1
    F2[key] = dates

# L_jk = 1 if team j plays as visitant in the date k in
# an external tournament (Copa Libertadores, Sudamericana, etc.)
L = {

    # Libertadores
    0  : [5],          # Independiente Medellín
    1  : [5, 9],       # Atlético Nacional
    11 : [15, 21, 23], # Junior
    13 : [15, 23],     # Deportes Tolima

    # Sudamericana
    2 : [29],          # Deportivo Cali
    4 : [17],          # La Equidad
    7 : [25],          # Rionegro
    8 : [5],           # Once Caldas
}

for i in range(n):
    if not i in L.keys():
        L[i] = []

for key in L.keys():
    dates = np.zeros(m)
    for value in L[key]:
        dates[value - 1] = 1
    L[key] = dates


# Best teams (based on 2019 1)
# Millonarios, Cali, Tolima, América, Nacional, Pasto, Junior & Unión Magdalena
b = 8
B = [18, 2, 13, 3, 1, 9, 11, 10]


# Worst teams (based on 2019 1)
# Santa Fe, Rionegro, Huila, Bucaramanga, Equidad, Jaguares, Alianza Petrolera, Envigado
w = 8
W = [19, 7, 12, 16, 4, 14, 15, 6]

# Others
# Medellín, Patriotas, Once Caldas, Cúcuta Deportivo
O = [0, 5, 8, 17] 


# Gains for day of matches and importance of the teams
e = {
     #          week          end
     #       B   W   -     B   W   - 
    'B' : [[.7, .2, .3], [ 1, .4, .5]],
    'W' : [[.3, .2, .4], [.5, .9, .6]],
    'O' : [[.6, .5, .5], [.7, .6, .6]],
}

z = {
    'D' : 4,
    'N' : 6
}

ext = [21, 29, 17, 25, 5, 9, 7, 11, 13, 15, 23, 27]
I = [ext.count(i - 1) for i in range(m)]

In [5]:
# Inizializate the variables

# X_ijk = 1 if team i plays against team j in date k
X = []

for i in T:
    row = []
    for j in T:
        row.append([None] * m)
    X.append(row)

for i in T:
    for j in T:
        for k in P:
            vname = '{} vs {} in date {}'.format(names[i], names[j], k + 1)
            X[i][j][k] = M.addVar(vtype = GRB.BINARY, name = vname)


# Decision variable (Y_jk = 1 if team j must play as visitor in date k and k + 1)
Y = [None] * n
for i in range(n):
    row = [None] * (m - 1)
    for k in range(m - 1):
        row[k] = M.addVar(lb = 0, name = 'y_{} {}'.format(i, k), vtype = GRB.BINARY)
    Y[i] = row


# Artificial variable
# Value = 0
phi = M.addVar(lb = 0, vtype = GRB.INTEGER, ub = 0);

# Ak = 1 if in date k there are local matches, otherwise is 0
A = [None] * m
for i in range(m):
    A[i] = M.addVar(vtype = GRB.BINARY, name = 'I_{}'.format(i))
    
# V_ij = 1 if team i and j are neighbors, 0 otherwise
V = np.zeros((n, n))

V[0, 1] = V[1, 0] = 1  # Medellín - Nacional
V[0, 4] = V[4, 0] = 1  # Medellín - Envigado
V[0, 5] = V[5, 0] = 1  # Medellín - Rionegro
V[1, 4] = V[4, 1] = 1  # Nacional - Envigado
V[1, 5] = V[5, 1] = 1  # Nacional - Rionegro
V[4, 5] = V[5, 4] = 1  # Envigado - Rionegro

V[2, 3] = V[3, 2] = 1  # Cali - América

V[ 4, 18] = V[18,  4] = 1  # Equidad - Millonarios
V[ 4, 19] = V[19,  4] = 1  # Equidad - Santa Fe
V[18, 19] = V[19, 18] = 1  # Millonarios - Santa Fe

V[11, 10] = V[10, 11] = 1  # Junior - Unión Magdalena
V[11, 14] = V[14, 11] = 1  # Junior - Jaguares
V[10, 14] = V[14, 10] = 1  # Unión Magdalena - Jaguares

In [6]:
# Restrictions

"""
1. All teams play once against each other
"""
M.addConstrs((quicksum(X[i][j][k] + X[j][i][k] for k in Q) == 1 for i in T for j in T if i != j))

"""
2. Each team must play the same number of matches as local as visitor
"""
M.addConstrs((quicksum(X[i][j][k] - X[j][i][k] for j in T if j != i for k in P) == 0 for i in T));

"""
3. Each team can play at most one match each date
"""
M.addConstrs((quicksum(X[i][j][k] + X[j][i][k] for j in T) + F2[i][k] <= 1 for i in T for k in P));

"""
4. Teams that share stadium cannot play as local the same date
"""
M.addConstrs((quicksum(X[i][h][k] + X[j][h][k] for h in T) <= 1 for i, j in S for k in P));

"""
5. Classic matches must be played only once
"""
M.addConstrs((X[i][j][c] + X[j][i][c] == 1 for i, j in C));

"""
6. Classic matches can not be repeated
"""
M.addConstrs((quicksum(X[i][j][k] for k in Q) + X[i][j][c] == 1 for i, j in C));

"""
7. Teams can not play in the forbidden dates
"""
M.addConstrs((quicksum(X[i][j][k - 1] + X[j][i][k - 1] for k in F[i]) == 0 for i in T for j in T if i != j));

In [7]:
# New restrictions [September 5]

"""
8. If Y_jk is 1, the team j must play as visitant in dates k and k + 1
"""
M.addConstrs((quicksum(X[i][j][k] + X[i][j][k + 1] for i in T) + L[j][k] + L[j][k + 1] <= 1 + Y[j][k] for j in T for k in P[:-1]));

"""
9. No more than phi dates as visitant for any team
"""
M.addConstrs((quicksum(Y[j][k] for k in P[:-1]) <= phi for j in T));

"""
10. No more than 3 dates as visitor for any team
"""
M.addConstrs((quicksum(quicksum(X[i][j][k1] for i in T) + L[j][k1] for k1 in range(k, k + 3)) <= 2 for j in T for k in P[:-2]));

In [8]:
# New restrictions [September 12]

"""
11. Half of the matches must be played before classic date
"""
M.addConstrs((quicksum(X[i][j][k] + X[j][i][k] for k in P[:c+1] for j in T) == n // 2 for i in T));

"""
12. Best teams
"""
M.addConstrs((quicksum(X[j][i][k] for j in B if j != i for k in Q) == b // 2 for i in T if not i in B));

M.addConstrs((quicksum(X[j][i][k] for j in B if j != i for k in Q) <= b // 2 for i in B));

M.addConstrs((quicksum(X[j][i][k] for j in B if j != i for k in Q) >= (b // 2) - 1 for i in B));

"""
13. Worst teams
"""
M.addConstrs((quicksum(X[i][j][k] for j in W if j != i for k in Q) == w // 2 for i in T if not i in W));

M.addConstrs(quicksum(X[i][j][k] for j in W if j != i for k in Q) <= w // 2 for i in W);

M.addConstrs(quicksum(X[i][j][k] for j in W if j != i for k in Q) >= (w // 2) - 1 for i in W);

In [9]:
# New restrictions [September 19]

"""
14. Classic matches can't be played in the first 6 dates of the tournament
"""
M.addConstrs(quicksum(X[i][j][k] + X[j][i][k] for k in P[7:]) == 2 for i, j in C);

"""
15. Classic matches must have at least 4 dates between them
"""
M.addConstrs(quicksum(X[i][j][k] + X[j][i][k] for k in P[c - 4 : c + 5]) == 1 for i, j in C);

"""
16. Half of the matches against best teams must be played before classical week 8
"""
M.addConstrs(quicksum(X[i][j][k] + X[j][i][k] for k in P[:c] for i in B) == b // 2 for j in T if not j in B);

M.addConstrs(quicksum(X[i][j][k] + X[j][i][k] for k in P[:c] for i in B) <= b // 2 for j in B);

M.addConstrs(quicksum(X[i][j][k] + X[j][i][k] for k in P[:c] for i in B) >= (b // 2) - 1 for j in B);

In [10]:
# New restrictions [October 10]

"""
17. If there are an international date, maximum 7 local matches can be played
"""
M.addConstrs(quicksum(X[i][j][k] for i in T for j in T if i != j) <= 10 - 3 * I[k] for k in Q);

"""
18. Boundes for Ak
"""
M.addConstrs(n * A[k] >= quicksum(X[i][j][k] for i in T for j in T if i != j) for k in P);

M.addConstrs(A[k] <= quicksum(X[i][j][k] for i in T for j in T if i != j) for k in P);

"""
19. Number of matches depending on what tournaments are played in date k
"""
M.addConstrs(quicksum(X[i][j][k] for i in T for j in T if i != j) >= 2 * (1 - I[k]) - 2 * (1 - A[k]) for k in Q);

In [11]:
# New restriction [October 17]

M.addConstr(quicksum(A[k] for k in P) <= 23);

In [12]:
# New restriction [November 7]

M.addConstrs(quicksum(X[i][j][k] * V[i][j] for k in Q for j in T) <= quicksum(V[i][j] for j in T) / 2 + 0.5 for i in T);

M.addConstrs(quicksum(X[i][j][k] * V[i][j] for k in Q for j in T) >= quicksum(V[i][j] for j in T) / 2 - 0.5 for i in T);

In [13]:
# Set an artificial objective

# obj = phi
# M.setObjective(obj, GRB.MINIMIZE)

In [14]:
# New objective [September 30]

# Best against best in weekday
BwB = quicksum(X[i][j][k] * e['B'][0][0] for i in B for j in B for k in P[0:m:2])
# Best against best in weekend
BnB = quicksum(X[i][j][k] * e['B'][1][0] for i in B for j in B for k in P[1:m:2])
# Best against worst in weekday
BwW = quicksum(X[i][j][k] * e['B'][0][1] for i in B for j in W for k in P[0:m:2])
# Best against worst in weekend
BnW = quicksum(X[i][j][k] * e['B'][1][1] for i in B for j in W for k in P[1:m:2])
# Best against others in weekday
BwO = quicksum(X[i][j][k] * e['B'][0][2] for i in B for j in O for k in P[0:m:2])
# Best against others in weekend
BnO = quicksum(X[i][j][k] * e['B'][1][2] for i in B for j in O for k in P[1:m:2])

# Worst against best in weekday
WwB = quicksum(X[i][j][k] * e['W'][0][0] for i in W for j in B for k in P[0:m:2])
# Worst against best in weekend
WnB = quicksum(X[i][j][k] * e['W'][1][0] for i in W for j in B for k in P[1:m:2])
# Worst against worst in weekday
WwW = quicksum(X[i][j][k] * e['W'][0][1] for i in W for j in W for k in P[0:m:2])
# Worst against worst in weekend
WnW = quicksum(X[i][j][k] * e['W'][1][1] for i in W for j in W for k in P[1:m:2])
# Worst against others in weekday
WwO = quicksum(X[i][j][k] * e['W'][0][2] for i in W for j in O for k in P[0:m:2])
# Worst against others in weekend
WnO = quicksum(X[i][j][k] * e['W'][1][2] for i in W for j in O for k in P[1:m:2])

# Other against best in weekday
OwB = quicksum(X[i][j][k] * e['O'][0][0] for i in O for j in B for k in P[0:m:2])
# Other against best in weekend
OnB = quicksum(X[i][j][k] * e['O'][1][0] for i in O for j in B for k in P[1:m:2])
# Other against worst in weekday
OwW = quicksum(X[i][j][k] * e['O'][0][1] for i in O for j in W for k in P[0:m:2])
# Other against worst in weekend
OnW = quicksum(X[i][j][k] * e['O'][1][1] for i in O for j in W for k in P[1:m:2])
# Other against other in weekday
OwO = quicksum(X[i][j][k] * e['O'][0][2] for i in O for j in O for k in P[0:m:2])
# Other against other in weekend
OnO = quicksum(X[i][j][k] * e['O'][1][2] for i in O for j in O for k in P[1:m:2])

obj = BwB + BnB + BwW + BnW + BwO + BnO + WwB + WnB + WwW + WnW + WwO + WnO + OwB + OnB + OwW + OnW + OwO + OnO
# obj = -phi + (1 / n ** 2)*(BwB + BnB + BwW + BnW + BwO + BnO + WwB + WnB + WwW + WnW + WwO + WnO + OwB + OnB + OwW + OnW + OwO + OnO)
M.setObjective(obj, GRB.MAXIMIZE);

In [ ]:
# Search for a feasible solution

# M.setParam(GRB.Param.TimeLimit, 3600 * 1.5);
M.optimize();

Optimize a model with 2933 rows, 12611 columns and 211463 nonzeros
Variable types: 0 continuous, 12611 integer (12610 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 2e+01]
Presolve removed 1590 rows and 4603 columns
Presolve time: 0.54s
Presolved: 1343 rows, 8008 columns, 81954 nonzeros
Variable types: 0 continuous, 8008 integer (8008 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   13492    1.2219983e+02   1.948639e+03   0.000000e+00      5s
   16035    1.2220000e+02   0.000000e+00   0.000000e+00      7s

Root relaxation: objective 1.222000e+02, 16035 iterations, 6.15 seconds
Total elapsed time = 13.36s
Total elapsed time = 15.91s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  122.20000    0  612          -  

  8601  7137  119.78479  267  400          -  122.10000      -   558 3424s
  9119  7533  121.87857  146  413          -  122.10000      -   546 3502s
  9552  7887  122.01069  140  382          -  122.10000      -   538 3579s
  9818  8085  121.95000  159  388          -  122.10000      -   541 3654s
 10213  8395  122.03883  142  416          -  122.10000      -   537 3737s
 10558  8665  121.06090  184  366          -  122.10000      -   535 3821s
 10899  8937  121.40198  208  376          -  122.10000      -   534 3913s
 11270  9221  121.23369  185  359          -  122.10000      -   532 3993s
 11554  9472  121.02561  189  383          -  122.10000      -   533 4069s
 11893  9712  121.72666  174  355          -  122.10000      -   532 4144s
 12251  9978 infeasible  161               -  122.10000      -   530 4212s
 12433 10148  119.80832  210  351          -  122.10000      -   530 4284s
 12768 10392  122.00139  132  405          -  122.10000      -   528 4339s
 13013 10596  119.73804  

 36951 29167  120.71321  174  375          -  122.10000      -   503 9557s
 37208 29398  121.56145  175  329          -  122.10000      -   503 9599s
 37540 29652  120.29361  216  324          -  122.10000      -   503 9645s
 37833 29859  122.05521  143  420          -  122.10000      -   502 9683s
 38115 30091 infeasible  250               -  122.10000      -   502 9720s
 38363 30274  122.06741  138  425          -  122.10000      -   502 9745s
 38516 30412  120.84042  186  339          -  122.10000      -   503 9771s
 38775 30585  121.74776  153  418          -  122.10000      -   503 9796s
 39093 30848  122.01364  156  418          -  122.10000      -   502 9820s
 39397 31082  121.69939  178  387          -  122.10000      -   502 9846s
 39638 31266  120.91580  195  363          -  122.10000      -   502 9871s
 39891 31445  122.09705  141  400          -  122.10000      -   502 9895s
 40036 31572  121.80238  169  390          -  122.10000      -   502 9922s
 40166 31701  119.93945  

 62780 49333  121.67677  182  378          -  122.10000      -   503 13326s
 63064 49559  121.91230  170  359          -  122.10000      -   503 13366s
 63363 49791 infeasible  192               -  122.10000      -   502 13409s
 63558 49929  121.95000  148  367          -  122.10000      -   503 13451s
 63799 50119  121.84414  152  347          -  122.10000      -   503 13492s
 64009 50277  122.08294  146  403          -  122.10000      -   503 13529s
 64200 50439  121.50682  211  340          -  122.10000      -   504 13568s
 64442 50604  121.29442  163  273          -  122.10000      -   504 13608s
 64721 50858 infeasible  205               -  122.10000      -   504 13648s
 65018 51096 infeasible  213               -  122.10000      -   503 13684s
 65342 51321  121.40375  209  351          -  122.10000      -   503 13722s
 65574 51526  121.21346  199  372          -  122.10000      -   503 13759s
 65786 51675  120.10236  189  381          -  122.10000      -   503 13796s
 66007 51839

 88776 69690  121.90134  164  431          -  122.10000      -   498 17302s
 89004 69870  122.07916  132  444          -  122.10000      -   498 17342s
 89258 70070  121.48098  202  324          -  122.10000      -   498 17381s
 89459 70221  121.89414  180  405          -  122.10000      -   498 17422s
 89527 70265  121.46195  198  354          -  122.10000      -   499 17463s
 89665 70353  121.96580  148  457          -  122.10000      -   500 17504s
 89868 70532  119.86822  184  335          -  122.10000      -   500 17546s
 90108 70722  121.52090  157  393          -  122.10000      -   500 17588s
 90317 70891  121.61221  159  396          -  122.10000      -   500 17631s
 90527 71038  122.08000  134  437          -  122.10000      -   500 17676s
 90678 71164  121.40520  168  413          -  122.10000      -   501 17717s
 90927 71362  122.04981  139  446          -  122.10000      -   501 17759s
 91114 71497  121.36009  172  388          -  122.10000      -   501 17803s
 91312 71647

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\Jamer\Anaconda3\lib\site-packages\ipykernel\iostream.py", line 383, in write
    if self.echo is not None:
KeyboardInterrupt


 106241 83223 infeasible  211               -  122.10000      -   514 20226s
 106425 83351  121.35582  163  399          -  122.10000      -   514 20259s
 106723 83571  121.39040  166  414          -  122.10000      -   514 20291s


KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\Jamer\Anaconda3\lib\site-packages\ipykernel\iostream.py", line 383, in write
    if self.echo is not None:
KeyboardInterrupt


In [ ]:
# Make files

import codecs

dats = []
for _ in T:
    row = []
    for k in P:
        row.append('')
    dats.append(row)

for i in T:
    for j in T:
        for k in P:
            if i != j and X[i][j][k].X == 1:
                dats[i][k] = 'L'
                dats[j][k] = 'V'
            if F2[i][k] == 1:
                if [3, 5, 6, 9, 10, 12, 14, 15, 16, 17, 18, 19].count(i) == 0:
                    dats[i][k] = 'FL'
                    if L[i][k] == 1:
                        dats[i][k] = 'FV'
                #else:
                #   dats[i][k] = 'C'


file = open('dates.txt', 'w')
for row in dats:

    file.write(','.join(row) + '\n')

file.close()

file2 = codecs.open('names.txt', 'w', 'utf-8')
file2.write('\n'.join(names) + '\n')
file2.close()
print(names)